# Character Level CNN Variational Autoencoder

In [1]:
import Corpus.gutenberg as corpus
from TextPreprocess.Tokenizer.Stanford import tokenize
from Utils.visual import hist, tally
from Utils.debug import dump
from Utils.generator import sliding_window, random_window, transform
from Utils.FS.file import save, load
from Utils.keras import compact_embedding
from Utils.misc import batch
from Utils.indexer import build_index, index_2_one_hot
from keras.preprocessing.sequence import pad_sequences
from keras_tqdm import TQDMNotebookCallback

Using TensorFlow backend.


In [2]:
#data = tokenize(corpus.raw())
data = corpus.gutenberg.words()

In [3]:
def char_generator():
    for word in data:
        for char in word:
            yield char

In [4]:
s2i, i2s, size = build_index(char_generator())

In [5]:
MAX_SEQ_LENGTH = max([len(word) for word in data])
MAX_SEQ_LENGTH = 64

In [6]:
NUM_SYMBOL = size
NUM_SYMBOL

94

In [7]:
def word_generator():
    for word in data:
            yield word

NUM_SAMPLE = len(list(word_generator()))
NUM_SAMPLE

2621613

In [57]:
from keras.layers import Input, Conv2D, Conv2DTranspose, Dense, Flatten, Dropout, Reshape, Embedding
from keras.models import Model, Sequential
from keras.regularizers import l2
import numpy as np
import tensorflow as tf

def custom_loss(y_true, y_pred):
    print(y_true.shape)
    '''Just another crossentropy'''
    #y_pred = T.clip(y_pred, epsilon, 1.0 - epsilon)
    #y_pred /= y_pred.sum(axis=-1, keepdims=True)
    #cce = T.nnet.categorical_crossentropy(y_pred, y_true)
    '''
    [np.average
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=y_true[i],
            logits=y_pred[i],
        )
     for i in y_true]
    '''
    return y_true - y_pred

def create_baseline(dropout=0, branching=5):
    
    activation = 'selu'
    padding = 'same'
    use_bias = False
    layer = 5
    dim = [http://104.196.106.86:8888/notebooks/C_Char_CNN_VAE_C-Copy1.ipynb#NUM_SYMBOL, 200, 100, 50, 25, 12]
    kernel = [2, 2, 2, 2, 2]
    strides = [1, 1, 1, 1, 1]
    
    inp = Input(shape=(MAX_SEQ_LENGTH,NUM_SYMBOL))
    #x = Embedding(NUM_SYMBOL, NUM_SYMBOL, weights=[np.eye(NUM_SYMBOL)], input_length=10, trainable=False)(inp)
    x = Reshape((1, MAX_SEQ_LENGTH, NUM_SYMBOL))(inp)
    for i in range(layer):
        x = Conv2D(dim[i+1], (1, kernel[i]), strides=(1, strides[i]), activation=activation, padding=padding, use_bias=use_bias)(x)
        
    for i in reversed(range(layer)):
        x = Conv2DTranspose(dim[i], (1, kernel[i]), strides=(1, strides[i]), activation=activation, padding=padding,use_bias=use_bias)(x)
        
    x = Reshape((MAX_SEQ_LENGTH, NUM_SYMBOL))(x)
    model = Model(inp, x)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam')
    return model

SyntaxError: invalid syntax (<ipython-input-57-9796d09821dd>, line 29)

In [58]:
from keras_tqdm import TQDMNotebookCallback

BATCH_SIZE = 1024
DROPOUT = 0.5
BRANCHING = 2

model = create_baseline(DROPOUT, BRANCHING)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 64, 94)            0         
_________________________________________________________________
reshape_17 (Reshape)         (None, 1, 64, 94)         0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 1, 64, 200)        37600     
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 1, 64, 100)        40000     
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 1, 64, 50)         10000     
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 1, 64, 25)         2500      
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 1, 64, 12)         600       
__________

In [59]:
from keras.utils.np_utils import to_categorical

def encode_word(word):
    unpad = batch(s2i, word)
    padded = pad_sequences([unpad], maxlen=MAX_SEQ_LENGTH, dtype='float32', padding='pre', truncating='pre', value=0.)
    one_hot = to_categorical(padded, num_classes=NUM_SYMBOL)
    return one_hot

def decode_word(one_hots):
    return ''.join([i2s(np.argmax(one_hot)) for one_hot in one_hots if np.argmax(one_hot) > 0])
    

def sample_generator(word_generator, batch_size = 64):
    sample = []
    for word in word_generator:
        sample.append(encode_word(word))
        if len(sample) == batch_size:
            train = np.array(sample)
            yield(train, train)
            sample = []

In [60]:
word_gen = word_generator()
next(sample_generator(word_gen, 2))[0].shape

(2, 64, 94)

In [61]:
from keras.callbacks import Callback

def testing(model):
    word_list = [
        'morning',
        'one',
        'two',
        'three',
        'apple',
        'cat',
        'dog',
        'extraordinary',
        'Philip',
        'LOL',
        'Jesus',
        'Mary'
    ]
    inp = np.array(batch(encode_word, word_list))
    return batch(decode_word, model.predict(inp))

class testSample(Callback):
    def on_epoch_end(self, batch, logs={}):
        print(testing(model))

In [62]:
word_gen = word_generator()
BATCH_SIZE = 64
model.fit_generator(
    sample_generator(word_gen, BATCH_SIZE),
    NUM_SAMPLE // BATCH_SIZE // 100,
    epochs=100,
    #class_weight=class_weight
    #workers=32,
    #use_multiprocessing=False,
    verbose=0, callbacks=[TQDMNotebookCallback(), testSample()]
)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


325/|/[loss: 0.000]  79%|| 325/409 [00:33<00:04, 17.15it/s]['orsing', 'one', 'tno', 'eree', 'aalle', 'at', 'og', 'eoeaaertinary', 'hiein', 'hia', 'eses', 'are']


Widget Javascript not detected.  It may not be installed or enabled properly.


['morning', 'one', 't-o', 'teree', 'aalle', 'aat', 'dog', 'ertraoreinaty', 'hilin', 'ean', 'esas', 'ary']


...

['morning', 'one', 'two', 'three', 'anile', 'cat', 'dou', 'eatraordinary', 'thilis', 'so', 'esus', 'oary']


Widget Javascript not detected.  It may not be installed or enabled properly.


['morning', 'one', 'two', 'three', 'alile', 'cat', 'dou', 'eatraordinary', 'thilis', 'weo', 'esus', 'fary']


Widget Javascript not detected.  It may not be installed or enabled properly.


KeyboardInterrupt: 

In [ ]:
#decode_word(model.predict(np.array([encode_word('testing')]))[0])